In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (10000, 784), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 128
num_layer1 = 1024
num_layer2 = 256
num_layer3 = 32

regularization_meta=0.001
keep_prob = 0.5

graph = tf.Graph()

def weights_variables(input_nodes_num, output_nodes_num):
    stddev = np.sqrt(3.0/(input_nodes_num + output_nodes_num))
    return tf.Variable(tf.truncated_normal([input_nodes_num,output_nodes_num], stddev=stddev))

def biases_variables(output_nodes_num):
    return tf.Variable(tf.zeros([output_nodes_num]))

def relu_layer(input_data, layer_weights, layer_biases, dropout=True):
    linear = tf.add(tf.matmul(input_data, layer_weights), layer_biases)
    relu = tf.nn.relu(linear)
    
    if dropout:
        return tf.nn.dropout(relu, keep_prob)
    else:
        return relu

def multi_layers(ds, dropout=True):
    layer1 = relu_layer(ds, weights1, biases1, dropout=dropout)
    layer2 = relu_layer(layer1, weights2, biases2, dropout=dropout)
    layer3 = relu_layer(layer2, weights3, biases3, dropout=dropout)
    
    logits = relu_layer(layer3, weights, biases, dropout=False)
    
    return logits

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights1 = weights_variables(image_size * image_size, num_layer1)
    biases1 = biases_variables(num_layer1)
    
    weights2 = weights_variables(num_layer1, num_layer2)
    biases2 = biases_variables(num_layer2)
    
    weights3 = weights_variables(num_layer2, num_layer3)
    biases3 = biases_variables(num_layer3)
    
    weights = weights_variables(num_layer3, num_labels)
    biases = biases_variables(num_labels)
    
    logits = multi_layers(tf_train_dataset, dropout=True)
    logits_valid = multi_layers(tf_valid_dataset, dropout=False)
    logits_test = multi_layers(tf_test_dataset, dropout=False)

    l2_loss = regularization_meta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.3, global_step, 6400, 0.95, staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

    init_op = tf.initialize_all_variables()

In [7]:
num_epochs = 50001

with tf.Session(graph=graph) as session:
    
    session.run(init_op)

    for epoch in range(num_epochs):
        offset = (epoch * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if(epoch % 1000 == 0):
            print('Mini Batch Loss at epoch %d: %f' % (epoch, l))
            print('Training Mini Batch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Mini Batch Loss at epoch 0: 3.373096
Training Mini Batch accuracy: 10.9%
Validation accuracy: 22.7%
Mini Batch Loss at epoch 1000: 1.069272
Training Mini Batch accuracy: 84.4%
Validation accuracy: 84.7%
Mini Batch Loss at epoch 2000: 0.952549
Training Mini Batch accuracy: 81.2%
Validation accuracy: 85.8%
Mini Batch Loss at epoch 3000: 0.974886
Training Mini Batch accuracy: 78.9%
Validation accuracy: 86.6%
Mini Batch Loss at epoch 4000: 0.659387
Training Mini Batch accuracy: 85.2%
Validation accuracy: 86.6%
Mini Batch Loss at epoch 5000: 0.665005
Training Mini Batch accuracy: 86.7%
Validation accuracy: 86.4%
Mini Batch Loss at epoch 6000: 0.874492
Training Mini Batch accuracy: 82.8%
Validation accuracy: 87.2%
Mini Batch Loss at epoch 7000: 0.723583
Training Mini Batch accuracy: 84.4%
Validation accuracy: 87.4%
Mini Batch Loss at epoch 8000: 0.659284
Training Mini Batch accuracy: 85.9%
Validation accuracy: 87.5%
Mini Batch Loss at epoch 9000: 0.727148
Training Mini Batch accuracy: 85.9%
